In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder

In [ ]:
import os
os.listdir('../Data/dataFiles/')

In [ ]:
df = pd.read_csv('../Data/dataFiles/df_merged_v2.csv', index_col=0)

In [ ]:
cols_2_drop =['Column','Unnamed: 0_x','action','court_outcome','establishment_address','establishment_status',\
             'inspection_id','long_lat_x','int_lat_x','int_long_x', 'int_long_lat_x',  'new_int_long', \
              'new_int_lat',  'level_0',  'index', 'Unnamed: 0_y', 'business_id', 'address', 'name',  'postal_code',\
              'state', 'long_lat_y', 'int_lat_y', 'int_long_y',  'int_long_lat_y', 'name_match', 'amount_fined',\
              'new_long_lat', 'latitude_y', 'longitude_y'
             ]

In [ ]:
def check_and_drop_cols(df, cols):
    for i in cols:
        if i in df.columns:
            df = df.drop(i,axis =1)
    return df

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df = check_and_drop_cols(df, cols_2_drop)

In [ ]:
df_v2 = df.copy()

In [ ]:
df_v2.shape

#### Get Open/Closing Times

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
for day in days:
    open_col_name = day + ' Start_time'
    close_time_col_name = day + ' Close_time'
    col_name = 'hours.' + day
    if col_name in df_v2.columns:
        print(col_name)
        df_v2[open_col_name] = df_v2[col_name].str[:2].str.replace(":", " ")
        df_v2[close_time_col_name]=df_v2[col_name].str[-5:].str.replace("-"," ").str[:2]


In [ ]:
df_v2.shape

In [ ]:
df_v2['inspection_date'] =pd.to_datetime(df_v2['inspection_date'], errors = 'coerce')
df_v2['insp_day']=df_v2['inspection_date'].dt.day
df_v2['insp_day_of_week']=df_v2['inspection_date'].dt.dayofweek


In [ ]:
df_v3 = df_v2.copy()


In [ ]:
cols_2_drop_new = ['establishment_name', 'infraction_details', 'categories']

In [ ]:
df_v3 = check_and_drop_cols(df_v3, cols_2_drop_new)

In [ ]:
df_v3['attributes.AcceptsInsurance']

In [ ]:
for i in df_v3.columns:
    a = pd.unique(df_v3[i])
    if (0 in a and 1 in a):
        print(f"{i} {a}")
        # break
        df_v3[i] = df_v3[i].fillna(value=0);

### Dummy Variables

In [ ]:
establishment_type_dummies = pd.get_dummies(df_v3['establishment_type'], prefix ='establishment_type')
ages_allowed_dummies = pd.get_dummies(df_v3['attributes.AgesAllowed'], prefix ='ages_allowed')
alcohol_dummies = pd.get_dummies(df_v3['attributes.Alcohol'], prefix ='Alcohol')
Noise_lvl_dummies = pd.get_dummies(df_v3['attributes.NoiseLevel'], prefix ='Noise_lvl')
attire_dummies = pd.get_dummies(df_v3['attributes.RestaurantsAttire'], prefix ='Attire')
smoking_dummies =pd.get_dummies(df_v3['attributes.Smoking'], prefix ='Smoking')
wifi_dummies = pd.get_dummies(df_v3['attributes.WiFi'], prefix ='WiFi')
city_dummies = pd.get_dummies(df_v3['city'], prefix ='city')
neighborhood_dummies = pd.get_dummies(df_v3['neighborhood'], prefix ='neighborhood')

In [ ]:
df_v4 = df_v3.copy()

df_v4['severity'] = df_v4['severity'].fillna(value='NA - Not Applicable')
le = LabelEncoder()
df_v4['label_severity']=le.fit_transform(df_v4.severity.values)



#### Get rid of hours

In [ ]:
cols_2_drop_3 =['hours.Friday','hours.Monday','hours.Saturday','hours.Sunday', \
              'hours.Thursday','hours.Tuesday','hours.Wednesday']
df_v4 = check_and_drop_cols(df_v4, cols_2_drop_3)

#### Convert opening/closing times to numeric

In [ ]:
days

In [ ]:
for day in days:
    start_time_col_name = day + ' Start_time'
    close_time_col_name = day + ' Close_time'
    df_v4[start_time_col_name] = pd.to_numeric(df_v4[start_time_col_name], errors ='coerce')
    df_v4[close_time_col_name] = pd.to_numeric(df_v4[close_time_col_name], errors ='coerce')


In [ ]:
cols_2_drop_4 =['attributes.AgesAllowed','attributes.Alcohol','attributes.NoiseLevel','attributes.RestaurantsAttire', \
              'attributes.Smoking','attributes.WiFi','city','neighborhood', 'establishment_type']

In [ ]:
df_v4 = check_and_drop_cols(df_v4, cols_2_drop_4)

#### fill price range na with means

In [ ]:
df_v4['attributes.RestaurantsPriceRange2'] = df['attributes.RestaurantsPriceRange2']
df_v4['attributes.RestaurantsPriceRange2'] = df_v4['attributes.RestaurantsPriceRange2'].fillna(value=np.mean(df_v4['attributes.RestaurantsPriceRange2']));

In [ ]:
start_days =['Monday Start_time', 'Tuesday Start_time', 'Wednesday Start_time',
       'Thursday Start_time', 'Friday Start_time', 'Saturday Start_time',
       'Sunday Start_time']

In [ ]:
df_v4[start_days].describe()

In [ ]:
df_v4[start_days] = df_v4[start_days].apply(lambda row: row.fillna(row.mean()), axis=1)



In [ ]:
end_days =['Monday Close_time', 'Tuesday Close_time', 'Wednesday Close_time',
       'Thursday Close_time', 'Friday Close_time', 'Saturday Close_time',
       'Sunday Close_time']

In [ ]:
df_v4[end_days] = df_v4[end_days].apply(lambda row: row.fillna(row.mean()), axis=1)


In [ ]:
df_v5 = pd.concat([df_v4,establishment_type_dummies, ages_allowed_dummies, alcohol_dummies,\
                  Noise_lvl_dummies, attire_dummies, smoking_dummies, wifi_dummies, city_dummies, neighborhood_dummies], axis =1)


In [ ]:
cols_2_drop_5 = df_v5.columns[df_v5.isna().any()].tolist()
cols_2_drop_5

In [ ]:
df_v5 = check_and_drop_cols(df_v5, cols_2_drop_5)

In [ ]:
df_v5.shape

In [ ]:
df_v5.columns

In [ ]:
'establishment_type' in df_v4.columns

In [ ]:
df_v5_exist = pd.read_csv('../Data/dataFiles/Cleaned_Data.csv', index_col=0)

In [ ]:
df_v5_exist.shape

In [ ]:
df_v4 = check_and_drop_cols(df_v4, cols_2_drop_5)
df_v4.to_csv('Statistical_analysis_data_1.csv')

In [ ]:
establishment_type_dummies